In [48]:
import numpy as np
import pandas as pd

In [49]:
funroi_dir = "../data/funROI_output/spcorr"
funroi_data = []
ii = 0
for networks in ["language", "events"]:
    for task1, effect1 in [
        ("language", "S"),
        ("language", "W"),
        ("language", "N"),
        ("events", "Sem_photo"),
        ("events", "Perc_photo"),
        ("events", "Sem_sent"),
        ("events", "Perc_sent"),
    ]:
        for task2, effect2 in [
            ("language", "S"),
            ("language", "W"),
            ("language", "N"),
            ("events", "Sem_photo"),
            ("events", "Perc_photo"),
            ("events", "Sem_sent"),
            ("events", "Perc_sent"),
        ]:
            dat = pd.read_csv(
                f"{funroi_dir}/spcorr_{ii:04d}/spcorr_summary.csv"
            )
            dat["network"] = networks
            effect1_, effect2_ = sorted([effect1, effect2])
            dat["effect1"] = effect1_
            dat["effect2"] = effect2_
            funroi_data.append(dat)
            ii = ii + 1
funroi_data = pd.concat(funroi_data)
funroi_data

,parcels,fisher_z,subject,network,effect1,effect2
0,1,1.051131,40,language,S,S
1,2,0.421091,40,language,S,S
2,3,0.421969,40,language,S,S
3,4,1.120930,40,language,S,S
4,5,1.407738,40,language,S,S
...,...,...,...,...,...,...
127,7,1.153502,76,events,Perc_sent,Perc_sent
128,8,0.908147,76,events,Perc_sent,Perc_sent
129,9,1.255465,76,events,Perc_sent,Perc_sent
130,10,0.122992,76,events,Perc_sent,Perc_sent


In [50]:
spm_ss_dir = "../data/spm_ss_results/parcels_spcorr"
spm_ss_data = []
for networks in ["language", "events"]:
    for task1, effect1 in [
        ("language", "S"),
        ("language", "W"),
        ("language", "N"),
        ("events", "Sem_photo"),
        ("events", "Perc_photo"),
        ("events", "Sem_sent"),
        ("events", "Perc_sent"),
    ]:
        for task2, effect2 in [
            ("language", "S"),
            ("language", "W"),
            ("language", "N"),
            ("events", "Sem_photo"),
            ("events", "Perc_photo"),
            ("events", "Sem_sent"),
            ("events", "Perc_sent"),
        ]:
            dat = pd.read_csv(
                f"{spm_ss_dir}/spcorr_parcels_{networks}_{task1}2{task2}_{effect1}2{effect2}/spm_bcc_data.EffectSize.csv"
            )
            dat["network"] = networks
            effect1_, effect2_ = sorted([effect1, effect2])
            dat["effect1"] = effect1_
            dat["effect2"] = effect2_
            spm_ss_data.append(dat)
spm_ss_data = pd.concat(spm_ss_data)
spm_ss_data

,ROI,Subject,Effects,Fisher transformed correlation coefficients,network,effect1,effect2
0,1,1,S/S,1.095165,language,S,S
1,1,2,S/S,1.051131,language,S,S
2,1,3,S/S,1.052649,language,S,S
3,1,4,S/S,0.895994,language,S,S
4,1,5,S/S,1.250540,language,S,S
...,...,...,...,...,...,...,...
127,11,8,Perc_sent/Perc_sent,0.092961,events,Perc_sent,Perc_sent
128,11,9,Perc_sent/Perc_sent,1.062734,events,Perc_sent,Perc_sent
129,11,10,Perc_sent/Perc_sent,0.883699,events,Perc_sent,Perc_sent
130,11,11,Perc_sent/Perc_sent,1.016943,events,Perc_sent,Perc_sent


In [51]:
sorted_subjects = np.sort(funroi_data["subject"].unique())
spm_ss_data["Subject"] = sorted_subjects[spm_ss_data["Subject"].values - 1]

In [52]:
# Average across (effect1, effect2) - spm_ss is assymetric when orthogonalized,
# so to average across the two directions.
funroi_data = (
    funroi_data[
        ["parcels", "network", "effect1", "effect2", "subject", "fisher_z"]
    ]
    .groupby(["parcels", "network", "effect1", "effect2", "subject"])
    .mean()
    .reset_index()
)
spm_ss_data = (
    spm_ss_data[
        [
            "ROI",
            "network",
            "effect1",
            "effect2",
            "Subject",
            "Fisher transformed correlation coefficients",
        ]
    ]
    .groupby(["ROI", "network", "effect1", "effect2", "Subject"])
    .mean()
    .reset_index()
)

In [53]:
data = pd.merge(
    spm_ss_data,
    funroi_data,
    left_on=["ROI", "Subject", "network", "effect1", "effect2"],
    right_on=["parcels", "subject", "network", "effect1", "effect2"],
)
data = data.rename(
    columns={
        "Fisher transformed correlation coefficients": "spm_ss FisherZ",
        "fisher_z": "funroi FisherZ",
    }
)[
    [
        "subject",
        "network",
        "effect1",
        "effect2",
        "ROI",
        "spm_ss FisherZ",
        "funroi FisherZ",
    ]
]
data["spm_ss FisherZ - funroi FisherZ"] = (
    data["spm_ss FisherZ"] - data["funroi FisherZ"]
)
data["abs_diff"] = np.abs(data["spm_ss FisherZ"] - data["funroi FisherZ"])
data.sort_values("abs_diff", ascending=False).head(50)

,subject,network,effect1,effect2,ROI,spm_ss FisherZ,funroi FisherZ,spm_ss FisherZ - funroi FisherZ,abs_diff
458,56,language,Perc_photo,Sem_photo,1,0.832862,0.832862,4.999815e-07,4.999815e-07
5240,76,events,Perc_sent,Sem_sent,10,0.066321,0.066321,4.997875e-07,4.997875e-07
2905,40,events,S,S,5,0.704891,0.704891,-4.997729e-07,4.997729e-07
5477,67,events,Perc_photo,Perc_sent,11,0.318188,0.318188,4.996502e-07,4.996502e-07
3450,68,events,Perc_photo,Perc_photo,6,0.456230,0.456230,4.995893e-07,4.995893e-07
3513,78,events,Perc_photo,W,6,0.672242,0.672242,4.995228e-07,4.995228e-07
4422,68,events,N,Sem_photo,8,1.028459,1.028459,-4.991249e-07,4.991249e-07
2623,70,language,Sem_photo,Sem_photo,4,0.613129,0.613129,4.990552e-07,4.990552e-07
2237,67,events,S,S,4,1.007177,1.007177,-4.990054e-07,4.990054e-07
831,57,events,Perc_sent,Perc_sent,2,0.121606,0.121606,-4.989816e-07,4.989816e-07
